# beef import to japan from USA Australia

In [ ]:

import pandas as pd
import pandas.io.sql as psql
import sqlite3


from datetime import datetime as dt
import time
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import re
import json
import inspect
import requests
import codecs
import numpy as np


import plotly.graph_objects as go


#['custom_2019_2021.db', 'codes4.db', 'ym_2019_2021.db']
# http://www.customs.go.jp/toukei/info/tsdl_e.htm
show_tables = "select tbl_name from sqlite_master where type = 'table'"
desc = "PRAGMA table_info([{table}])"
conn = \
    sqlite3.connect(':memory:')
cursor = conn.cursor()

In [ ]:
# hs code,country,
attach = 'attach "../input/japantradestatistics3/codes4.db" as code'
cursor.execute(attach)

# import hs,country code as pandas
tmpl = "{hs}_{lang}_df =  pd.read_sql('select * from code.{hs}_{lang}',conn)"
for hs in ['hs2','hs4','hs6','hs6','hs9']:
    for lang in ['jpn','eng']:
        exec(tmpl.format(hs=hs,lang=lang))        
        
# country table: country_eng,country_jpn
country_eng_df = pd.read_sql('select * from code.country_eng',conn)
country_eng_df['Country']=country_eng_df['Country'].apply(str)
country_jpn_df = pd.read_sql('select * from code.country_jpn',conn)
country_jpn_df['Country']=country_jpn_df['Country'].apply(str)


In [ ]:
y_range = '2019_2021'
attach = 'attach "../input/japantradestatistics3/ym_{y_range}.db" as ym_{y_range}'
cursor.execute(attach.format(y_range=y_range))




In [ ]:
hs = 'hs4'
hscode = '0201' # 牛肉
exp_imp=2

In [ ]:
# 直近の状況をチェック
ym='202104'
sql_tmpl = """
select Country,sum(Value) as Value from ym_{y_range}
where {hs}="{hscode}" and exp_imp={exp_imp} and  ym='{ym}'
group by Country
order by Value desc
"""[1:-1]
sql = sql_tmpl.format(hs=hs,hscode=hscode,exp_imp=exp_imp,ym=ym,y_range=y_range)
df = pd.read_sql(sql,conn)

total = df['Value'].sum()
df['Country'] = df['Country'].astype('str') # Country が、数字なので、文字型に変更
df = pd.merge(df,country_eng_df,on='Country')
df['percent'] = 100*(df['Value']/total)

df['name']=df['Country_name'] + ': ' + df['Country']# index に Country_name 設定

In [ ]:
xdf = df.head(5).sort_values('percent')
fig = go.Figure(go.Bar(
            x=xdf['percent'],
            y=xdf['name'],
            orientation='h'))

fig.show()

In [ ]:
# HS コードを調べたりする utility 
class util():
    def __init__(self):
        print(dt.now())
        
    # hs code から　説明を取得する
    def hs_name_get(self,hs_code):
        hs = len(hs_code)
        if hs not in (2,4,6,9):
            print('HS コードの長さがまちがっています。 ' + str(hs))
        hs = str(hs)
        print(hs_code)
        text = 'hs' + hs + '_eng_df.query(' +"'"+ 'hs' + hs + '=="' + hs_code + '"' + "')"
        df = eval(text)
        print(df['hs' + hs + '_name'].values[0])
        text = 'hs' + hs + '_jpn_df.query(' +"'"+ 'hs' + hs + '=="' + hs_code + '"' + "')"
        df = eval(text)
        print(df['hs' + hs + '_name'].values[0])
        
    def hs_url(self,hs_code,exp_imp=2,yyyy_mm='2020_4'):
        # 輸出 https://www.customs.go.jp/yusyutu/index.htm
        # 輸入 https://www.customs.go.jp/tariff/index.htm
        hs = hs_code[0:2]
    
        if exp_imp == 1:
            ex = 'yusyutu'
        else:
            ex = 'tariff'
        
        tmpl = 'https://www.customs.go.jp/{ex}/{yyyy_mm}/data/print_j_{hs}.htm'
        print(tmpl.format(ex=ex,yyyy_mm=yyyy_mm,hs=hs))
    
    # とりあえず適当に ym で、scatter する
    def scatter(self,df,code,name,xtype='Country'):
        text = xtype + '=="' + code + '"'
        x = list(df.query(text)['ym'])
        y = list(df.query(text)['Value'])
        return(go.Scatter(x=x,
               y=y,name=name,line=dict(width=5)))


ut = util()

In [ ]:
# ５国の比較

sql_tmpl = """
select ym,Country,sum(Value) as Value from ym_{y_range}
where {hs}="{hscode}" and exp_imp={exp_imp} and 
Country in ('304','601','302','606','305')
group by ym,Country
order by ym,Country
"""[1:-1]
sql = sql_tmpl.format(hs=hs,hscode=hscode,exp_imp=exp_imp,y_range=y_range)
df = pd.read_sql(sql,conn)

df['Country'] = df['Country'].astype('str')
df['ym'] = df['ym'].astype(str)
df['ym'] = 'ym' + df['ym']
df =pd.merge(df,country_eng_df,on='Country')


In [ ]:
fig = go.Figure()
fig.add_trace(ut.scatter(df,'304','USA'))
fig.add_trace(ut.scatter(df,'601','Australia'))
fig.add_trace(ut.scatter(df,'302','Canada'))
fig.add_trace(ut.scatter(df,'606','New Zealand'))
fig.add_trace(ut.scatter(df,'305','Mexico'))
fig.show()

In [ ]:
%%time

sql_tmpl = """
select ym,Country,sum(Value) as Value from ym_{y_range}
where {hs}="{hscode}" and exp_imp={exp_imp} and 
ym > 201909 and
Country in ('304','601')
group by ym,Country
order by ym,Country
"""[1:-1]
sql = sql_tmpl.format(hs=hs,hscode=hscode,exp_imp=exp_imp,y_range=y_range)
df = pd.read_sql(sql,conn)
df['Country'] = df['Country'].astype('str')

df =pd.merge(df,country_eng_df,on='Country')


df['ym'] = df['ym'].astype('str')
df['ym'] = 'ym' + df['ym']

def func(Country,name):
    text = 'Country=="' + Country + '"'
    x = list(df.query(text)['ym'])
    y = list(df.query(text)['Value'])
    return(go.Scatter(x=x,
               y=y,name=name,line=dict(width=5)))

In [ ]:


fig = go.Figure()   # グラフの初期化

t1 = func('304','USA')
t2 = func('601','Australia')


fig.add_trace(t1.update(marker=dict(size=12)))
fig.add_trace(t2.update(marker=dict(size=12)))

# コメントをつける 数字の場合はそのまま行けるけど、文字列の場合は、リスト上の位置（添字）で指定します。

fig.update_layout(width=1080,height=1080,paper_bgcolor='rgba(248,148,172,1)',
        
                  legend=dict(x=0,font_size=18,bgcolor='#eee'))


fig.update_layout(
    title={
        'text': "牛肉輸入上位は、オーストラリアとアメリカ。今はアメリカ優勢",
        'font_size':24,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.add_layout_image(
    dict(
        source="https://zanjibar.github.io/flag_of_japan.jpg?",
        xref="paper", yref="paper",
        x=0.03, y=1.02,
        sizex=0.1, sizey=0.1,
        xanchor="right", yanchor="bottom"
    )
)

fig.add_annotation(x=2, y=11000000,
            text="2019.10 - 2021.05 <br>HS 0201",
            align='left',
            font_size = 24,
            showarrow=False)


fig.show()

"""

fig.add_annotation(x=0, y=12700000,
            text="アメリカ",
            font_size = 18,
            font_color='red',
            showarrow=False)


fig.add_annotation(x=0, y=10300000,
            text="カナダ",
            font_size = 18,
            font_color='blue',       
            showarrow=False)

fig.add_annotation(x=1, y=11640000,
            text="カナダがアメリカを抜く",arrowhead=1,
            showarrow=True)


fig.add_annotation(x=3, y=11680000,
            text="日米貿易協定で、アメリカ逆転",arrowhead=1,yshift=12,
            showarrow=True)

fig.add_annotation(x=7, y=14000000,
            text="コロナでアメリカの豚肉大手スミスフィールドが操業停止",
            showarrow=False)


fig.update_layout(width=1080,height=1080,paper_bgcolor='rgba(248,148,172,1)',
                  title = dict(text='豚肉輸入上位は、カナダとアメリカでデッドヒート',font_size=24),
                  legend=dict(x=0,font_size=18,bgcolor='#eee'))


                  plot_bgcolor='rgba(0,0,0,0)',
"""

''
